In [ ]:
#libraries 
import random
import threading
import time
import xml.etree.ElementTree as ET

# Define the ITstudent class
class ITstudent:
    def __init__(self, name, student_id, program, courses, marks):
        self.name = name
        self.student_id = student_id
        self.program = program
        self.courses = courses
        self.marks = marks
        
#calculating average mark
    def calculate_average(self):
        total_marks = sum(self.marks)
        average = total_marks / len(self.marks)
        return average

#checking if student passed or failed
    def is_passed(self):
        average = self.calculate_average()
        if average >= 50:
            return "Pass"
        else:
            return "Fail"

# Define the Buffer class
class Buffer:
    def __init__(self, max_size):
        self.max_size = max_size
        self.buffer = []
        self.semaphore = threading.Semaphore(1)
        self.empty = threading.Semaphore(self.max_size)
        self.full = threading.Semaphore(0)

    def insert(self, item):
        self.empty.acquire()
        self.semaphore.acquire()
        self.buffer.append(item)
        self.semaphore.release()
        self.full.release()

    def remove(self):
        self.full.acquire()
        self.semaphore.acquire()
        item = self.buffer.pop(0)
        self.semaphore.release()
        self.empty.release()
        return item

# Define the Producer class
class Producer(threading.Thread):
    def __init__(self, buffer):
        threading.Thread.__init__(self)
        self.buffer = buffer

    def run(self):
        for i in range(1, 11):
            student = generate_student(self)
            xml_file = create_xml_file(student, i)
            self.buffer.insert(i)
            time.sleep(random.randint(1, 3))
            
#generating student random
    def generate_name(self):
        names = random.choice(["DLAMINI SINDY","DLAMINI SENZO", "MHLONGO SINDISIWE N.","NCUBE LIFA T.","NGCOBO NKULULEKO",
"NHLABATSI PHUMZILE T","NKONYANE SIBEKELWELOKUHLE L", "NYEMBE SIBONGILE", "SHIBA NCOBILE", "SITHOLE WANDILE", "TSABEDZE TENINA","SIMELANE LUNGISAN",
 "MKHONTA SENZO","DLAMINI GCINILE", "MMEMA MPUMELELO"])
        return names
    
#generating courses
    def generate_courses(self):
        courses = ['Database and Design', 'Computer Networking', 'Security',
                   'Data Structures and Algorithms', 'Software Engineering',
                   'Modern Operating Systems', 'Web Technology and Development',
                   'Data Mining', 'Graphics']
        return random.sample(courses, random.randint(5,5))


# Function to generate a random student
def generate_student(self):
    name = self.generate_name()
    student_id = str(random.randint(10000000, 99999999))
    program = "Information Technology"
    courses = self.generate_courses()
    marks = [random.randint(0, 100) for _ in range(len(courses))]
    return ITstudent(name, student_id, program, courses, marks)

# Function to create an XML file for a student
def create_xml_file(student, index):
    root = ET.Element("Student")
    name = ET.SubElement(root, "Name")
    name.text = student.name
    student_id = ET.SubElement(root, "StudentID")
    student_id.text = student.student_id
    program = ET.SubElement(root, "Program")
    program.text = student.program
    courses = ET.SubElement(root, "Courses")
    for i in range(len(student.courses)):
        course = ET.SubElement(courses, "Course")
        course.text = student.courses[i]
        mark = ET.SubElement(course, "Mark")
        mark.text = str(student.marks[i])
    tree = ET.ElementTree(root)
    xml_file = f"student{index}.xml"
    tree.write(xml_file)
    return xml_file

# Define the Consumer class
class Consumer(threading.Thread):
    def __init__(self, buffer):
        threading.Thread.__init__(self)
        self.buffer = buffer

    def run(self):
        while True:
            item = self.buffer.remove()
            xml_file = f"student{item}.xml"
            student = parse_xml_file(xml_file)
            average = student.calculate_average()
            pass_fail = student.is_passed()
            print_student_info(student, average, pass_fail)
            time.sleep(random.randint(1, 3))

# Function to parse an XML file and create an ITstudent object
def parse_xml_file(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    name = root.find("Name").text
    student_id = root.find("StudentID").text
    program = root.find("Program").text
    courses = []
    marks = []
    for course in root.find("Courses"):
        courses.append(course.text)
        marks.append(int(course.find("Mark").text))
    return ITstudent(name, student_id, program, courses, marks)

# Function to print student information
def print_student_info(student, average, pass_fail):
    print("__________________________________\n")
    print("Student Name:", student.name)
    print("Student ID:", student.student_id)
    print("Program:", student.program)
    print("\n")
    print("Courses and Marks:")
    print("------------------\n")
    for i in range(len(student.courses)):
        print("-", student.courses[i], ":", student.marks[i])
    print("\n")
    print("Average Mark:", average)
    print("Pass/Fail:", pass_fail)
    print("__________________________________\n")

# Create the buffer
buffer = Buffer(10)

# Create the producer and consumer threads
producer = Producer(buffer)
consumer = Consumer(buffer)

# Start the threads
producer.start()
consumer.start()

# Wait for the threads to finish
producer.join()
consumer.join()


__________________________________

Student Name: NGCOBO NKULULEKO
Student ID: 56425107
Program: Information Technology


Courses and Marks:
------------------

- Modern Operating Systems : 54
- Computer Networking : 52
- Web Technology and Development : 27
- Security : 54
- Data Structures and Algorithms : 94


Average Mark: 56.2
Pass/Fail: Pass
__________________________________

__________________________________

Student Name: DLAMINI SINDY
Student ID: 28204555
Program: Information Technology


Courses and Marks:
------------------

- Modern Operating Systems : 20
- Graphics : 46
- Web Technology and Development : 15
- Software Engineering : 33
- Security : 40


Average Mark: 30.8
Pass/Fail: Fail
__________________________________

__________________________________

Student Name: SHIBA NCOBILE
Student ID: 30852108
Program: Information Technology


Courses and Marks:
------------------

- Computer Networking : 16
- Graphics : 44
- Web Technology and Development : 80
- Database and 